# Input Filenames Here

`season_data_file` is the data file from which to create features (could be only fraction of regular season in future). `team_features_output_file` is the filename where the features for each team will be saved.

In [67]:
season_data_file = 'data/RegularSeasonDetailedResults.csv' 
team_features_output_file = 'data/team_features.csv'

In [68]:
import pandas as pd
import numpy as np

season_df = pd.read_csv(season_data_file)
teams_df = pd.read_csv('data/Teams.csv')
team_name = dict(zip(teams_df['Team_Id'].values, teams_df['Team_Name'].values))
team_id = dict(zip(teams_df['Team_Name'].values, teams_df['Team_Id'].values))

# Create dictionaries for boolean masks
seasons = season_df['Season'].unique()
in_season = dict(zip(seasons, [season_df['Season'] == i for i in seasons]))

team_ids = teams_df['Team_Id'].unique()
is_winner = dict(zip(team_ids, [season_df['Wteam'] == i for i in team_ids]))
is_loser = dict(zip(team_ids, [season_df['Lteam'] == i for i in team_ids]))
did_play = dict(zip(team_ids, [(season_df['Wteam'] == i) | (season_df['Lteam'] == i) for i in team_ids]))# winner or loser

is_valid = season_df['Wteam'] != 0

def avg(feature): # pass string for feature (without W or L to get a function back that calculates average for the season)
    def avg_feat(team):
        f = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'W'+feature], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'L'+feature]])
        return np.round(f.mean(), decimals=2)
    avg_feat.__name__ = 'avg_'+feature
    return avg_feat

def std(feature): # pass string for feature (without W or L to get a function back that calculates stdev for the season)
    def std_feat(team):
        f = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'W'+feature], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'L'+feature]])
        return np.round(f.std(), decimals=2)
    std_feat.__name__ = 'std_'+feature
    return std_feat


# ADD NEW TEAM FEATURES HERE (at bottom of cell below)

Add your new features specific to a team here, see tutorial at bottom of notebook.  You can create features that generate simple averages or standard deviations of features in the detailed data using the avg and std functions above.  What you add here to `teamfList` are the features that will be appended to the data file.

In [74]:
team_features_df = pd.read_csv(team_features_output_file)
team_features_df['Team_Id'].tail()

4727    1464
4728    1464
4729    1464
4730    1464
4731    1464
Name: Team_Id, dtype: int64

In [66]:
def wins(team):  
    return season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid].shape[0]

# Four factors
def efg(team):
    fgm = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfgm'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfgm']])
    fgm3 = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfgm3'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfgm3']])
    fga = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfga'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfga']])
    efg = (fgm+0.5*fgm3)/fga
    return np.round(efg.mean(), decimals=2)

def to_p(team):
    to = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wto'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lto']])
    fga = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfga'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfga']])
    o_r = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wor'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lor']])
    fta = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfta'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfta']])
    possessions = 0.96*(fga-o_r-to+0.475*fta)
    to_p = to/possessions
    return np.round(to_p.mean(), decimals=2)

def orp(team):
    o_r = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wor'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lor']])
    dr_op = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Ldr'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wdr']])
    orp = o_r/(o_r + dr_op)
    return np.round(orp.mean(), decimals=2)
    
def ftr(team):
    fta = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfta'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfta']])
    fga = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfga'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfga']])
    ftr = fta/fga
    return np.round(ftr.mean(), decimals=2)

In [8]:
season_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [9]:
teams_df.head()

,Team_Id,Team_Name
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M


In [42]:
print(ex['Team_Id'])
print(ex['Season'])

1148
2006


In [63]:
season_df.loc[13945]

Season    2006
Daynum      18
Wteam     1148
Wscore      76
Lteam     1128
Lscore      67
Wloc         H
Numot        0
Wfgm        25
Wfga        57
Wfgm3        9
Wfga3       22
Wftm        17
Wfta        25
Wor         16
Wdr         24
Wast        16
Wto         11
Wstl         4
Wblk         4
Wpf         15
Lfgm        26
Lfga        60
Lfgm3        5
Lfga3       18
Lftm        10
Lfta        14
Lor         12
Ldr         21
Last         8
Lto         12
Lstl         5
Lblk         3
Lpf         19
Name: 13945, dtype: object

In [64]:
o_r = pd.concat([season_df.loc[is_winner[ex['Team_Id']] & in_season[ex['Season']] & is_valid, 'Wor'], season_df.loc[is_loser[ex['Team_Id']] & in_season[ex['Season']] & is_valid, 'Lor']])
o_r

13945    16
14374     9
14557    15
14804    17
15236     9
15785    15
16105    14
16350     8
16427    11
16674    18
16746    10
16992    14
17656    13
17710     6
17900     6
18027     9
18166    11
18264    10
14039    19
14151    13
14696     9
15084    20
15409    13
15950    16
16046    14
17121    18
17365    14
17518    12
18447    21
dtype: int64

In [65]:
dr_op = pd.concat([season_df.loc[is_winner[ex['Team_Id']] & in_season[ex['Season']] & is_valid, 'Ldr'], season_df.loc[is_loser[ex['Team_Id']] & in_season[ex['Season']] & is_valid, 'Wdr']])
dr_op

13945    21
14374    14
14557    19
14804    24
15236    15
15785    21
16105    15
16350    17
16427    24
16674    23
16746    16
16992    20
17656    22
17710    16
17900    20
18027    12
18166    17
18264    19
14039    21
14151    29
14696    27
15084    32
15409    20
15950    23
16046    26
17121    33
17365    18
17518    20
18447    32
dtype: int64

In [40]:
ex = team_features_df.loc[614]
print(ex)

Unnamed: 0.1             614
Unnamed: 0.1             614
Team_Id                 1148
Season                  2006
team_name       Central Conn
wins                      18
avg_fgm                24.86
std_fgm                 4.73
avg_fga                   56
std_fga                 8.48
avg_fgm3                7.93
std_fgm3                2.53
avg_fga3               20.62
std_fga3                5.45
avg_ftm                14.83
std_ftm                  6.3
avg_fta                20.55
std_fta                 7.68
avg_or                  13.1
std_or                  4.05
avg_dr                 21.17
std_dr                  4.13
avg_ast                14.76
std_ast                 3.83
avg_to                 12.83
std_to                  3.27
avg_stl                 6.59
std_stl                  2.6
avg_blk                 2.34
std_blk                 1.32
avg_pf                 15.45
std_pf                  3.91
ftr                     0.38
efg                     0.52
Name: 614, dty

# Append your new function to teamfList below!

In [23]:
teamfList = []
teamfList.append(efg)
#teamfList.append(wins)
#for f in ['fgm', 'fga', 'fgm3', 'fga3', 'ftm', 'fta', 'or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf']:
#    teamfList.append(avg(f))
#    teamfList.append(std(f))
print("Team Features added = {0}".format([f.__name__ for f in teamfList]))

Team Features added = ['efg']


In [24]:
#USE THESE TWO LINES TO REGENERATE ALL THE TEAM FEATURES
#team_features_df = pd.DataFrame([[team_ID, season] for team_ID in team_ids for season in seasons], columns = ['Team_Id', 'Season'])
#team_features_df['team_name'] = team_features_df['Team_Id'].map(team_name)

#USE THIS LINE TO ADD TO THE EXISTING TEAM FEATURES
team_features_df = pd.read_csv(team_features_output_file)

for feature in teamfList:
    team_features_df[feature.__name__] = team_features_df.apply(feature, axis=1)

team_features_df = team_features_df.replace(np.nan, 0)
team_features_df.to_csv(team_features_output_file, encoding='ascii')
team_features_df.tail()

,Unnamed: 0,Unnamed: 0.1,Team_Id,Season,team_name,wins,avg_fgm,std_fgm,avg_fga,std_fga,...,avg_to,std_to,avg_stl,std_stl,avg_blk,std_blk,avg_pf,std_pf,ftr,efg
4727,4727,4727,1464,2011,Youngstown St,7,24.71,3.79,59.75,6.74,...,12.14,3.51,6.21,2.13,4.82,2.63,20.18,5.22,0.30,0.48
4728,4728,4728,1464,2012,Youngstown St,14,23.79,3.54,55.31,6.39,...,11.38,3.96,6.28,2.53,5.31,2.42,16.55,4.49,0.31,0.51
4729,4729,4729,1464,2013,Youngstown St,14,23.90,4.05,56.34,8.07,...,11.72,4.16,6.34,2.86,5.31,2.12,17.72,4.15,0.37,0.49
4730,4730,4730,1464,2014,Youngstown St,12,25.38,4.09,55.90,7.67,...,11.28,3.02,6.38,2.78,2.48,1.72,19.03,4.34,0.41,0.52
4731,4731,4731,1464,2015,Youngstown St,8,25.03,3.80,56.66,7.12,...,11.72,3.27,5.90,2.76,1.93,1.67,18.62,5.52,0.37,0.51


# Input Filenames for Generating Features for the Games

`training_data_file` is the data file we want to use as training data (could include part of regular season in addition to tourney in future). `team_features_input_file` is the input file we want to use for the team features (generated above--default is `team_features_output_file`, i.e. what was used for output above).  `training_data_output_file` is the filename for the training data with all the games.

In [29]:
training_data_file = 'data/data.csv' # use TourneyDetailedResults.csv if regenerating from scratch
team_features_input_file = team_features_output_file
training_data_output_file = 'data/data.csv'

# Randomizing the Winning Team and Adding Features

We first randomly chooses the winner to be team 1 or team 0.  The result is stored in `tourney_df('Winner')` and the ID corresponding to team0 and team1 are in `tourney_df('team0')` and `tourney_df('team1')`, respectively.  We also add all the features in `data/team_features.csv` for team0 and team1.

In [30]:
tourney_df = pd.read_csv(training_data_file)

Ngames = tourney_df.shape[0]
print("{0} tournament games in the dataset".format(Ngames))

np.random.seed(0) # seed so we always get same answer
tourney_df['team0Win'] = np.random.randint(2,size=Ngames)

def team0(game):
    if game['team0Win'] == 1:
        return game['Wteam']
    else:
        return game['Lteam']

def team1(game):
    if game['team0Win'] == 1:
        return game['Lteam']
    else:
        return game['Wteam']

team_features_df = pd.read_csv(team_features_input_file, index_col=0)
seasons = tourney_df['Season'].unique()
in_season = dict(zip(seasons, [team_features_df['Season'] == i for i in seasons]))

def feature01(feature):
    def mod(game):
        f0 = team_features_df.loc[(team_features_df['Team_Id'] == game['team0']) & in_season[game['Season']], feature.__name__].values[0]
        f1 = team_features_df.loc[(team_features_df['Team_Id'] == game['team1']) & in_season[game['Season']], feature.__name__].values[0]
        return pd.Series({feature.__name__+'0':f0, feature.__name__+'1':f1})
    return mod

847 tournament games in the dataset


In [32]:
# UNCOMMENT THESE LINES IF YOU WANT TO REGENERATE THE TRAINING DATA FILE FROM SCRATCH
#for feature in [team0, team1]:
#    tourney_df[feature.__name__] = tourney_df.apply(feature, axis=1)

# first add all the features for individual teams
for feature in teamfList:
    tourney_df = pd.concat([tourney_df, tourney_df.apply(feature01(feature), axis=1)], axis=1)
    
tourney_df.head()

,Unnamed: 0,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,...,std_blk0,std_blk1,avg_pf0,avg_pf1,std_pf0,std_pf1,ftr0,ftr1,efg0,efg1
0,0,2003,134,1421,92,1411,84,N,1,32,...,1.72,1.65,18.30,19.10,4.56,3.69,0.53,0.38,0.50,0.49
1,1,2003,136,1112,80,1436,51,N,0,31,...,2.01,1.72,17.75,15.90,2.99,4.05,0.39,0.36,0.52,0.49
2,2,2003,136,1113,84,1272,71,N,0,31,...,2.94,3.16,19.41,18.76,3.25,4.34,0.47,0.40,0.52,0.50
3,3,2003,136,1141,79,1166,73,N,0,29,...,2.22,2.35,17.27,20.97,3.12,4.81,0.35,0.48,0.57,0.57
4,4,2003,136,1143,76,1301,74,N,1,27,...,1.63,2.49,17.10,18.67,3.74,4.21,0.35,0.39,0.52,0.53


# Add game features here

In addition to having features for each team separately, e.g., avg field goals made, we might try features for how two teams relate to one another, i.e., a feature that requires knowledge of the specific two teams in a game.  Add them in the cell below and update gamefList in the cell below that:

In [33]:
def gamefeature(game): # add here any features specific to a game, i.e., how to teams interact
    pass

# Append your new function to gamefList below!

In [34]:
gamefList=[] # features specific to a pair of teams (rather than to an individual team, e.g., seed difference, but not e.g. team seed)
#gamefList.append(gamefeature)
print("Game features added = {0}".format([f.__name__ for f in gamefList]))

Game features added = []


In [35]:
# now add features specific to a game (pair of teams)
for feature in gamefList:
    tourney_df[feature.__name__] = tourney_df.apply(feature, axis=1)
    
tourney_df.to_csv(training_data_output_file, encoding='ascii')
tourney_df.head()

,Unnamed: 0,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,...,std_blk0,std_blk1,avg_pf0,avg_pf1,std_pf0,std_pf1,ftr0,ftr1,efg0,efg1
0,0,2003,134,1421,92,1411,84,N,1,32,...,1.72,1.65,18.30,19.10,4.56,3.69,0.53,0.38,0.50,0.49
1,1,2003,136,1112,80,1436,51,N,0,31,...,2.01,1.72,17.75,15.90,2.99,4.05,0.39,0.36,0.52,0.49
2,2,2003,136,1113,84,1272,71,N,0,31,...,2.94,3.16,19.41,18.76,3.25,4.34,0.47,0.40,0.52,0.50
3,3,2003,136,1141,79,1166,73,N,0,29,...,2.22,2.35,17.27,20.97,3.12,4.81,0.35,0.48,0.57,0.57
4,4,2003,136,1143,76,1301,74,N,1,27,...,1.63,2.49,17.10,18.67,3.74,4.21,0.35,0.39,0.52,0.53


# Generate Features for Submission Data

The cells below take the games Kaggle wants us to submit, and generates the corresponding features for them like we do above for the training data.  `submission_data_output_file` is the name of the output file to write the submission data for our model to load and predict.

In [36]:
import pandas as pd
import numpy as np
submission_data_with_features_file = 'data/SampleSubmissionWithFeatures.csv'

In [37]:
print("Adding following Team Features to {0}: {1}".format(submission_data_with_features_file, [f.__name__ for f in teamfList]))
print("Adding following Game Features to {0}: {1}".format(submission_data_with_features_file, [f.__name__ for f in gamefList]))

Adding following Team Features to data/SampleSubmissionWithFeatures.csv: ['efg']
Adding following Game Features to data/SampleSubmissionWithFeatures.csv: []


In [38]:
sample_df = pd.read_csv(submission_data_with_features_file, index_col=0) #should be SampleSubmission.csv if generating from scratch

def get_season(Id):
    return np.int64(Id.split("_")[0])
def get_team0(Id):
    return np.int64(Id.split("_")[1])
def get_team1(Id):
    return np.int64(Id.split("_")[2])

# UNCOMMENT THESE LINES IF YOU WANT TO REGENERATE THE SUBMISSION FEATURES FROM SCRATCH
#sample_df['Season'] = sample_df['Id'].apply(get_season)
#sample_df['team0'] = sample_df['Id'].apply(get_team0)
#sample_df['team1'] = sample_df['Id'].apply(get_team1)

# first add all the features for individual teams
for feature in teamfList:
    sample_df = pd.concat([sample_df, sample_df.apply(feature01(feature), axis=1)], axis=1)
    
# now add features specific to a game (pair of teams)
for feature in gamefList:
    tourney_df[feature.__name__] = tourney_df.apply(feature, axis=1)
    
sample_df.to_csv(submission_data_with_features_file, encoding='ascii')
sample_df.head()

,Unnamed: 0.1,Id,Pred,Season,team0,team1,wins0,wins1,avg_fgm0,avg_fgm1,...,std_blk0,std_blk1,avg_pf0,avg_pf1,std_pf0,std_pf1,ftr0,ftr1,efg0,efg1
0,0,2012_1104_1124,0.5,2012,1104,1124,21,26,23.62,26.09,...,2.68,2.98,18.41,17.64,4.01,4.30,0.38,0.37,0.49,0.53
1,1,2012_1104_1125,0.5,2012,1104,1125,21,26,23.62,27.91,...,2.68,1.68,18.41,18.27,4.01,4.01,0.38,0.40,0.49,0.56
2,2,2012_1104_1140,0.5,2012,1104,1140,21,23,23.62,27.97,...,2.68,1.72,18.41,18.45,4.01,4.44,0.38,0.38,0.49,0.52
3,3,2012_1104_1143,0.5,2012,1104,1143,21,24,23.62,26.42,...,2.68,1.39,18.41,14.39,4.01,3.69,0.38,0.35,0.49,0.52
4,4,2012_1104_1153,0.5,2012,1104,1153,21,24,23.62,25.06,...,2.68,2.74,18.41,15.15,4.01,3.21,0.38,0.30,0.49,0.48


# Tutorial for adding a new feature

All functions to populate a feature should take a game, i.e., a row of `tourney_df`,  

In [11]:
game = tourney_df.loc[0]
print(game)

Season       2003
Daynum        134
Wteam        1421
Wscore         92
Lteam        1411
Lscore         84
Wloc            N
Numot           1
Wfgm           32
Wfga           69
Wfgm3          11
Wfga3          29
Wftm           17
Wfta           26
Wor            14
Wdr            30
Wast           17
Wto            12
Wstl            5
Wblk            3
Wpf            22
Lfgm           29
Lfga           67
Lfgm3          12
Lfga3          31
Lftm           14
Lfta           31
Lor            17
Ldr            28
Last           16
            ...  
std_fta0      8.2
std_fta1     9.02
avg_or0     13.17
avg_or1     12.28
std_or0      4.86
std_or1      3.73
avg_dr0      24.8
avg_dr1     23.17
std_dr0      5.55
std_dr1      5.44
avg_ast0     14.2
avg_ast1    13.03
std_ast0      4.4
std_ast1     4.79
avg_to0     15.23
avg_to1     16.21
std_to0      4.03
std_to1         5
avg_stl0     6.43
avg_stl1     7.07
std_stl0     2.31
std_stl1     3.14
avg_blk0     2.23
avg_blk1        3
std_blk0  

If it's a feature specific to a team, name it 0 or 1, and access team0's and/or team1's ID using the corresponding column.  `team_name` is a dictionary that takes a team ID and returns the name, and `team_id` is a dictionary that takes a team name and returns the ID.  You can use these to match up data you find elsewhere.

The function should return the value for your new feature.  As a simple example, to make a feature with the team's name, 

In [18]:
def name0(game):
    ID = game['team0']
    
    # Examples of using the dictionaries
    name = team_name[ID] 
    print(team_id[name]) 
    
    return name

name0(tourney_df.loc[0])

1411


'TX Southern'

# More Realistic Example

I had to look at different pandas examples/tutorials, but this might get you started.  For `wins0`, I wanted to be able to return the number of games `team0` won in the regular season.  What we actually have is a list of games, each of which have a winning team.  So we need to find all the games where `team0` was the winner, and add them all up.  So we start with the column that has all the IDs of the winning teams for each game.

In [19]:
wteam = season_df['Wteam']
print(wteam.head())

0    1104
1    1272
2    1266
3    1296
4    1400
Name: Wteam, dtype: int64


Now we make a boolean array that has the same length as `wteam` above, with `True` if the winning team is `team0` (which we take to be 1106 in this example):

In [20]:
games_where_1106_won = season_df['Wteam'] == 1106
print(games_where_1106_won.head())

0    False
1    False
2    False
3    False
4    False
Name: Wteam, dtype: bool


In [21]:
games_1106_won = wteam[games_where_1106_won]
print(games_1106_won.head())

613     1106
1095    1106
1584    1106
1714    1106
1873    1106
Name: Wteam, dtype: int64


But this has games for all seasons mixed together, so we have to choose the right season.  Say the game for which we're trying to make the feature happend in 2012.  Then we need another mask like we used above to separate the games where 1106 won, but this time for the season:

In [22]:
in_2012 = season_df['Season'] == 2012
print(in_2012.head())

0    False
1    False
2    False
3    False
4    False
Name: Season, dtype: bool


We can put boolean masks together like this:

In [23]:
games_1106_won_in_2012 = wteam[games_where_1106_won & in_2012]
print(games_1106_won_in_2012.head())

45628    1106
46766    1106
46845    1106
47078    1106
47248    1106
Name: Wteam, dtype: int64


For convenience, I've made a dictionary `in_season` that takes a year, and gives you the corresponding boolean mask for that year.  Using that notation, and finally also using the `is_valid` mask, which ensures that the game is valid (WTeam != 0), we have 

In [24]:
games_1106_won_in_2012 = wteam[games_where_1106_won & in_season[2012] & is_valid]
print(games_1106_won_in_2012.head())

45628    1106
46766    1106
46845    1106
47078    1106
47248    1106
Name: Wteam, dtype: int64


We can now get the length of the final array of filtered values using `shape`, and taking `[0]`, which gives the number of rows.  A minor final detail is that all these new variables make copies of slices of the dataframes, so it's slightly more efficient to do things without making copies with the `loc` function, which explains the one-liners in the wins0 and wins1 functions above: 

In [25]:
print(games_1106_won_in_2012.shape[0])
print(season_df.loc[(season_df['Wteam'] == 1106) & in_season[2012] & is_valid].shape[0])

10
10


# Additional Features

Someother features to consider on the historic record of one Seed versus another. And the spread betting record. Here is an example of generate feature from data/TourneySeedBetting.csv

**TBD: find resource for 2016 league**

In [26]:
tourney_seeddf = pd.read_csv('data/TourneySeedBetting.csv')
tourney_seeddf.head()

### Wseed gives the Seed for the winning team, Lseed gives the Seed for the losing team. 
### meanProb gives the Probability of the team with the smaller seed wins, numProb 
### gives the number of games this probability was calculated based on
### spreadbet give the spreadbetting from covers on the winning team versus losing team.

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wseed,Lseed,meanProb,numProb,spreadbet
0,2011,134,1155,70,1412,52,N,0,12,12,0.50,1,0
1,2011,134,1421,81,1114,77,N,1,16,16,0.50,1,0
2,2011,135,1427,70,1106,61,N,0,16,16,0.50,1,0
3,2011,135,1433,59,1425,46,N,0,11,11,0.50,1,0
4,2011,136,1139,60,1330,58,N,0,8,9,0.51,124,1


In [27]:
def meanprob(game):
    return tourney_seeddf[(tourney_seeddf['Wteam'] == game['Wteam']) & (tourney_seeddf['Lteam']==game['Lteam']) & (tourney_seeddf['Season']==game['Season'])]['meanProb']

In [28]:
game = tourney_df[tourney_df['Season']==2011].iloc[0]
print game.head()
meanprob(game)

Season    2011
Daynum     134
Wteam     1155
Wscore      70
Lteam     1412
Name: 512, dtype: object


0    0.5
Name: meanProb, dtype: float64